In [7]:
def gaussian_eliminationU(A, b, verbose=False):
    # To ensure that arrays are stored in double precision.
    A = A.astype(np.float64)
    b = b.astype(np.float64)
     
    # size of solution vector / the square matrix A
    n=len(b) # or   n, n = A.shape
        
    # check sizes of A and b match appropriately
    nb=len(b)
    n, m = A.shape
    if n != m:
        raise ValueError(f'A is not a square matrix! {A.shape}')
    if n != nb:
        raise ValueError(f'shapes of A and b do not match! {A.shape} {b.shape}')
    
    if verbose:
        print('starting system\n', A, b)

    # perform forward elimination
    for i in range(n):  
        # eliminate column i
        if verbose:
            print(f'eliminating column {i}')
        
        # check diagonal
        if np.isclose(A[i, i], 0.0):
            raise ValueError(f'A has zero on diagonal! A[{i}, {i}] = 0')

        # row j <- row j - (a_{ji} / a_{ii}) row i
        for j in range(i+1, n):
            if verbose:
                print(f'row {j} <- row {j} - {A[j, i] / A[i, i]} row {i}')
            factor = A[j, i] / A[i, i]
            for k in range(0, n):
                A[j, k] = A[j, k] - factor * A[i, k]
            b[j] = b[j] - factor * b[i]
        

    return upper_triangular_solve(A, b)  

In [8]:
import numpy as np

def gaussian_eliminationC(A, b):
    n = A.shape[0]
    for i in range(n):
        # Eliminate
        for k in range(i + 1, n):
            c = -A[k, i]/A[i, i]
            A[k, i] = 0
            for j in range(i + 1, n):
                A[k, j] += c*A[i, j]
            b[k] += c*b[i]
    # Back substitution
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:]))/A[i, i]
    return x

In [15]:
import numpy as np
# Doesnt actually work
def gaussian_eliminationPC(A, b):
    n = A.shape[0]
    for i in range(n):
        # Find pivot row
        max_element = np.abs(A[i, i])
        max_row = i
        for k in range(i + 1, n):
            if np.abs(A[k, i]) > max_element:
                max_element = np.abs(A[k, i])
                max_row = k
        # Swap pivot row
        if max_row != i:
            A[[i, max_row]] = A[[max_row, i]]
            b[[i, max_row]] = b[[max_row, i]]
        # Eliminate
        for k in range(i + 1, n):
            c = -A[k, i]/A[i, i]
            A[k, i] = 0
            for j in range(i + 1, n):
                A[k, j] += c*A[i, j]
            b[k] += c*b[i]
    # Back substitution
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:]))/A[i, i]
    return x


A = np.array([[-10, 2, 0, 67], [-2, 50, -77, 1.e-5], [1, 7, 30, 8], [-10, -7, 0.001, 80]])
b = np.array([1, 2, 9, 0])

# numpy linear solvers
x0 = np.linalg.solve(A,b)
#x0 = np.linalg.inv(A).dot(b)
print("Solution by numpy solver:", x0)

x = gaussian_eliminationPC(A, b)
print("Gaussian elimination: ",x)
print("Residual: ", np.matmul(A,x)-b)

print(sum([i**2 for i in b-(A@x)])) # Residual or Total residual
print(np.sqrt(sum([i**2 for i in b-(A@x)]))) # Magnitude of residual

A = np.array([[2, 2, 1], [0, 1, -1], [0, 0, -1]])
b = np.array([100, 50, 50])



Solution by numpy solver: [0.9244595  0.31826746 0.15668124 0.14340388]
Gaussian elimination:  [0.78188447 0.27769044 0.14442538 0.12333528]
Residual:  [0. 0. 0. 0.]
0.0
0.0


In [11]:
# Note, it needs the upper triangular code from the other file
def upper_triangular_solve(A, b):
    """
    Solve the system  A x = b  where A is assumed to be lower triangular,
    i.e. A(i,j) = 0 for j > i, and the diagonal is assumed to be nonzero,
    i.e. A(i,i) != 0.
    
    The code checks that A is lower triangular and converts A and b to
    double precision before computing.

    ARGUMENTS:  A   lower triangular n x n array
                b   right hand side column n-vector

    RETURNS:    x   column n-vector solution
    """

    # we should take care to ensure that arrays are stored with the correct type - float!
    A = A.astype(np.float64)
    b = b.astype(np.float64)
      
    # check sizes of A and b match appropriately
    nb=len(b)
    n, m = A.shape
    if n != m:
        raise ValueError(f'A is not a square matrix! {A.shape}')
    if n != nb:
        raise ValueError(f'shapes of A and b do not match! {A.shape} {b.shape}')
    
    # check A is upper triangular
    for i in range(n):
        for j in range(0,i):
            if not np.isclose(A[i, j], 0.0):
                raise ValueError(f'A is not upper triangular! {A[i, j]}')

    # checks whether A has zero diagonal element
    for i in range(n):
        if np.isclose(A[i, i], 0.0):
            raise ValueError(f'A[{i}, {i}] is zero')
    
    #create a new array to store the results
    x = np.empty_like(b)
    
    # perform backwards substitution
    x[n-1] = b[n-1] / A[n-1, n-1]
    for i in range(2,n+1):
        x[n-i] = b[n-i] / A[n-i, n-i]
        for j in range(n-i+1, n):
            x[n-i] = x[n-i] - A[n-i,j]*x[j] / A[n-i, n-i]
        
    return x

def Gaussian_elimination_pivoting(A, b, verbose=False):
    # To ensure that arrays are stored in double precision.
    A = A.astype(np.float64)
    b = b.astype(np.float64)
     
    # size of solution vector / the square matrix A
    n=len(b) # or   n, n = A.shape
        
    # check sizes of A and b match appropriately
    nb=len(b)
    n, m = A.shape
    if n != m:
        raise ValueError(f'A is not a square matrix! {A.shape}')
    if n != nb:
        raise ValueError(f'shapes of A and b do not match! {A.shape} {b.shape}')
    
    if verbose:
        print('starting system\n', A, b)
  
    # perform forward elimination
    for i in range(n):          
        # find the index of the maximal value in column i on or below
        # the diagonal of A
        maximum = abs(A[i,i])
        max_index = i
        for j in range(i+1,n):
            if abs(A[j,i]) > maximum :
                maximum = abs(A[j,i])               
                max_index = j   
                                       
        
        # swap two max_indexs: i and max_index[i]
        temp = b[i]
        b[i] = b[max_index]
        b[max_index] = temp
        for j in range(n):
            temp = A[i,j]
            A[i,j] = A[max_index,j]
            A[max_index,j] = temp  
            
        
        # check diagonal
        if np.isclose(A[i, i], 0.0):
            raise ValueError(f'A has zero on diagonal! A[{i}, {i}] = 0') 

        # row j <- row j - (a_{ji} / a_{ii}) row i
        for j in range(i+1, n):
            if verbose:
                print(f'row {j} <- row {j} - {A[j, i] / A[i, i]} row {i}')
            factor = A[j, i] / A[i, i]
            for k in range(0, n):
                A[j, k] = A[j, k] - factor * A[i, k]
            b[j] = b[j] - factor * b[i]
        
    return upper_triangular_solve(A, b)

A = np.array([[-10, 2, 0, 67], [-2, 50, -77, 1.e-5], [1, 7, 30, 8], [-10, -7, 0.001, 80]])
b = np.array([1, 2, 9, 0])

# numpy linear solvers
x0 = np.linalg.solve(A,b)
#x0 = np.linalg.inv(A).dot(b)
print("Solution by numpy solver:", x0)

x = Gaussian_elimination_pivoting(A, b)
print("Gaussian elimination: ",x)
print("Residual: ", b-np.matmul(A,x))


Solution by numpy solver: [0.9244595  0.31826746 0.15668124 0.14340388]
Gaussian elimination:  [0.9244595  0.31826746 0.15668124 0.14340388]
Residual:  [0.00000000e+00 1.77635684e-15 1.77635684e-15 0.00000000e+00]
